# Метрические алгоритмы. Практика

В этом домашнем задании вы будете решать задачу классификации бутылок вина по различным характеристикам.

## Импорт библиотек, установка константных значений

In [60]:
import pandas as pd
import numpy as np

In [61]:
RANDOM_STATE = 42
TRAIN_SIZE = 0.75

In [62]:
rng = np.random.default_rng(RANDOM_STATE)

## Загрузка данных

In [63]:
from sklearn.datasets import load_wine

data = load_wine(as_frame=True)

X = data.data
y = data.target

## Задание 1

Посмотрите на количество классов и количество объектов каждого класса в датасете.

**Вопрос**:  
Сколько классов в задаче?

In [64]:
# ваш код здесь
y.value_counts()

target
1    71
0    59
2    48
Name: count, dtype: int64

## Задание 2

Мы имеем дело с многоклассовой классификацией. Кроме того, классы не очень хорошо сбалансированы, поэтому для оценки качества модели метрика *accuracy* не подойдет.

Разбейте данные на тренировочную и тестовую части:  
тестовая часть - 25% от всех данных, зафиксируйте `random_state = RANDOM_STATE`.

In [65]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state = RANDOM_STATE)    # ваш код здесь

**Вопрос:**

Все ли признаки в данных одного масштаба?  
Проверьте это, выведя основные числовые характеристики матрицы `X_train` методом `describe` из библиотеки `pandas`.

По полученной таблице числовых характеристик определите, какой признак измеряется в сотнях?  
(если вариантов несколько, выберите признак с наибольшим средним значением).

In [66]:
# ваш код здесь
X_train.describe().T

,count,mean,std,min,25%,50%,75%,max
alcohol,133.0,12.972857,0.829993,11.03,12.29,12.99,13.69,14.83
malic_acid,133.0,2.386842,1.098905,0.89,1.64,1.90,3.17,5.80
ash,133.0,2.362180,0.280606,1.36,2.21,2.36,2.54,3.23
alcalinity_of_ash,133.0,19.433835,3.467312,10.60,17.20,19.40,21.50,30.00
magnesium,133.0,100.759398,14.999571,70.00,88.00,98.00,108.00,162.00
total_phenols,133.0,2.277068,0.645696,0.98,1.70,2.23,2.80,3.88
flavanoids,133.0,2.021203,1.005537,0.34,1.20,2.14,2.88,5.08
nonflavanoid_phenols,133.0,0.363534,0.126923,0.13,0.26,0.34,0.45,0.66
proanthocyanins,133.0,1.608647,0.576964,0.42,1.25,1.56,1.96,3.58
color_intensity,133.0,5.017594,2.202516,1.74,3.25,4.80,6.13,10.80


## Задание 3

KNN требует того, чтобы все признаки были одного масштаба, поэтому масштабируйте данные при помощи `StandardScaler`.

Напоминаем, что обучать метод нужно только по тренировочным данным, а применять и к трейну, и к тесту.

После применения `StandardScaler` преобразуйте `X_train` и `X_test` к типу `pd.DataFrame`, названия новых объектов оставьте `X_train` и `X_test`.

In [67]:
from sklearn.preprocessing import StandardScaler

# ваш код здесь\
scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

Обучите KNN с параметрами по умолчанию на тренировочных данных и сделайте предсказание на тесте.

In [68]:
from sklearn.neighbors import KNeighborsClassifier

# ваш код здесь
model = KNeighborsClassifier()

model.fit(X_train, y_train)

KNeighborsClassifier()

Будем измерять качество модели по метрике weighted $f1$-score.

Чтобы выбрать тип усреднения (micro, macro, weighted) в функции `f1_score` необходимо задать этот тип в гиперпараметре `average`.

Вычислите $f1$-score на тестовых данных.

**Вопрос:**

Чему равен $f1$-score на тестовых данных?

In [69]:
from sklearn.metrics import f1_score

# ваш код здесь
pred = model.predict(X_test)

f1_weighted = f1_score(y_test, pred, average='weighted')
print(f1_weighted)

0.9550512333965844


## Задание 4

Попробуем улучшить модель.

Подберите оптимальное количество соседей (`n_neighbors`) из диапазона *от 3 до 30 с шагом 2* и веса соседей (`weights`):  
`uniform`, `distance` по кросс-валидации с тремя фолдами на тренировочных данных.

Используйте `GridSearchCV` и метрику `f1_weighted`.

In [70]:
from sklearn.model_selection import GridSearchCV

# ваш код здесь
params = {'n_neighbors' : np.arange(3, 30, 2), 'weights': ['uniform', 'distance']} 

gs = GridSearchCV(KNeighborsClassifier(), params, cv=3, scoring='f1_weighted')

gs.fit(X_train, y_train)

print(gs.best_score_) 
print(gs.best_params_) 

0.9623290498688744
{'n_neighbors': 19, 'weights': 'distance'}


Возьмите best_estimator_, полученный при обучении GridSearchCV и с помощью него  
сделайте предсказание на тесте и вычислите метрику `f1_weighted`.

In [71]:
# ваш код здесь
model2 = KNeighborsClassifier(n_neighbors=19, weights='distance')

model2.fit(X_train, y_train)

pred = model2.predict(X_test)

f1_weighted2 = f1_score(y_test, pred, average='weighted')
print(f1_weighted2)

0.9550512333965844


**Вопрос:**

Удалось ли при помощи подбора гиперпараметров улучшить качество модели на тестовых данных?

## Задание 5

Выведите на экран матрицу ошибок.

Используйте модель с подобранными при помощи `GridSearch` гиперпараметрами.


**Вопрос:**  
По этой матрице определите, какие классы между собой путает модель?

In [72]:
from sklearn.metrics import confusion_matrix

# ваш код здесь
print(confusion_matrix(y_test, pred))

[[15  0  0]
 [ 1 16  1]
 [ 0  0 12]]


Столбцы - реальные значения классов 0, 1, 2. Строки - прогноз. Из второго столбца видим, что все реальные объекты класса 1 были отнесены к классу 1. Из 1-го столбца видим, что 1 объект класса 0 был ошибочно отнесен к классу 1, а из 3-го столбца видим, что 1 объект класса 2 был ошибочно отнесен к классу 1. Итого модель путает классы 0 и 1, а также 2 и 1

In [73]:
from sklearn.metrics import classification_report

print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97        15
           1       1.00      0.89      0.94        18
           2       0.92      1.00      0.96        12

    accuracy                           0.96        45
   macro avg       0.95      0.96      0.96        45
weighted avg       0.96      0.96      0.96        45



## Бонус (эксперименты с LSH)

Скопируйте все функции из [ноутбука в уроке "Быстрый поиск соседей"](https://colab.research.google.com/drive/181MMOcTnzdMVzJr0pWzqtEG0-BV9BIHH).

In [74]:
# обычный поиск с KNN
def knn_search(query, data, k=5):

    dists = np.sqrt(np.sum((data - query) ** 2, axis=1))  # вычисляем расстояния от объекта query до всех точек датасета
    inds = np.argsort(dists)  # сортируем по возрастанию расстояний
    inds_k = inds[:k]         # берем top-k точек с наименьшими расстояниями
    
    return data[inds_k], dists[inds_k]

In [75]:
# ваш код здесь
def generate_hyperplanes(data, bucket_size=16):
    m = data.shape[0]            # число объектов
    n = data.shape[1]            # число признаков
    b = m // bucket_size         # количество корзин
    h = int(np.log2(b))          # количество гиперплоскостей
    H = rng.normal(size=(h, n))  # гиперплоскости, заданные своими нормалями
    return H

In [76]:
def hamming_hash(data, hyperplanes):
    b = len(hyperplanes)
    hash_key = (data @ hyperplanes.T) >= 0
    
    dec_vals = np.array([2 ** i for i in range(b)], dtype=int)
    hash_key = hash_key @ dec_vals
    
    return hash_key

In [77]:
def locality_sensitive_hash(data, hyperplanes):
    hash_vals = hamming_hash(data, hyperplanes)
    hash_table = {}
    for i, v in enumerate(hash_vals):
        if v not in hash_table:
            hash_table[v] = set()
        hash_table[v].add(i)
        
    return hash_table

In [78]:
def approx_knn_search(query, data, k=5, bucket_size=16):
    candidates = set()

    hyperplanes = generate_hyperplanes(data)
    hash_table = locality_sensitive_hash(data, hyperplanes) # формируем хеш-таблицу по датасету

    query_hash = hamming_hash(query, hyperplanes)
    if query_hash in hash_table:
        candidates = candidates.union(hash_table[query_hash])
    candidates = np.stack([data[i] for i in candidates], axis=0) # находим кандидатов = объекты, попадающие с query в одну корзину

    return knn_search(query, candidates, k=k) # с помощью KNN ищем ближайших соседей только из найденных кандидатов

При помощи `knn_search` найдите ближайших соседей к вину `X_test.iloc[0]` в **тренировочных** данных.

Обратите внимание, что функция `knn_search` принимает на вход `np.array`, а не `pd.DataFrame`. Поэтому переведите аргументы в `np.array`, приписав к необходимому объекту $X$: `X.values`.

In [79]:
%%time

# ваш код здесь
q = np.reshape(X_test.iloc[0], ((1, 13)))

neighbors, dists = knn_search(q, X_train.values)  
for i, (neighbor, dist) in enumerate(zip(neighbors, dists)):
    print(f"top {i + 1}: dist = {dist}")

top 1: dist = 1.7873320988662762
top 2: dist = 1.9809624052592747
top 3: dist = 2.1601016426727613
top 4: dist = 2.270974564253457
top 5: dist = 2.3117955237515453
CPU times: total: 0 ns
Wall time: 1 ms


Выведите на экран признаки объекта `X_test.iloc[0]` и признаки ближайшего найденного соседа.

In [80]:
# ваш код здесь
X_test.iloc[0]

alcohol                         0.806832
malic_acid                      0.651425
ash                             0.707637
alcalinity_of_ash              -1.225688
magnesium                       1.019911
total_phenols                   0.657478
flavanoids                      1.007035
nonflavanoid_phenols           -1.530581
proanthocyanins                 0.089343
color_intensity                 0.037556
hue                             0.002381
od280/od315_of_diluted_wines    1.031337
proline                         0.333672
Name: 19, dtype: float64

In [81]:
pd.DataFrame(neighbors, columns=X_train.columns).iloc[0]

alcohol                         1.508275
malic_acid                      1.464385
ash                             0.528778
alcalinity_of_ash              -1.804684
magnesium                       1.822958
total_phenols                   1.123849
flavanoids                      1.017018
nonflavanoid_phenols           -1.293323
proanthocyanins                 0.820043
color_intensity                 0.037556
hue                            -0.297198
od280/od315_of_diluted_wines    1.265107
proline                         0.055633
Name: 0, dtype: float64

**Вопрос:**

Можно ли сказать, что в тренировочных данных есть вино, почти такое же как `X_test.iloc[0]`? (все признаки почти одинаковые)

Какое расстояние между объектом запроса и первым ближайшим соседом?

**нет, расстояние не равно 0**

**расстояние до ближайшего соседа равно 1,79**

Теперь найдите ближайшего соседа при помощи `approx_knn_search`.

In [83]:
%%time

# ваш код здесь

q = np.reshape(X_test.iloc[0].values, ((1, 13)))

neighbors, dists = approx_knn_search(q, X_train.values)

for i, (neighbor, dist) in enumerate(zip(neighbors, dists)):
    print(f"top {i + 1}: dist = {dist}")

TypeError: unhashable type: 'numpy.ndarray'

Ближайший сосед при помощи KNN+LSH может быть найден не точно или не с первого запуска.  
Запустите последнюю ячейку несколько раз и убедитесь, что ближайший сосед находится верно за несколько запусков.

**Вопрос:**

Запустите `knn_search` и `appox_knn_search` несколько раз и сравните время запусков. Какой из подходов в этой задаче работает быстрее?